* Filter:
    - Importance: High => Low
    - Medium questions only

## Flatten

* Clarification questions:
    - What type of data does the array contain? Some approach only applies to certain data types
    - How many levels of nesting can this array have? If there are thousands-of-levels of nesting, recursion might not be a good idea given its big upfront memory footprint.
    - Should we return a new array or we mutate the existing array?
        * would use splice
    - Can we assume valid input, i.e. an array. Normally the answer is "yes", so you don't have to waste your time doing defensive programming.
    - Does the environment the code runs on has ES6+ support? The environment determines what methods/native APIs you have access to.


In [ ]:
/**
 * @param {Array<*|Array>} value
 * @return {Array}
 * 
 * returns a new array
 * 
 * no matter how deep, we want to traverse it
 * definitely recursive
 *  - if we encounter an array, we traverse it
 *  - definitely have to check for sparsity
 */

// recursive
export default function flatten(value) {
  const result = [];

  const traverse = (arr) => {
    for (const element of arr) {
      if (Array.isArray(element)) {
        traverse(element);
      } else {
        result.push(element);
      }
    }
  }

  traverse(value);
  return result;
}

/*
 * iterative version
 * we use a stack instead
 * and reverse it at the end
 * so this is only around O(n) time complexity
*/
export default function flatten(value) {
  const res = [];
  const copy = [...value];

  while (copy.length) {
    const item = copy.pop();
    if (Array.isArray(item)) {
      copy.push(...item);
    } else {
      res.push(item);
    }
  }

  return res.reverse();
}

## Promise.all()

* use `forEach` so that you can pass in an async function
    - this allows you to await on the element concurrently on all items since `forEach` is synchronous
* new Array(arrLen) is much faster than Array.from({ length: arrLen })
* you don't need a while loop or anything to wait for counter to become 0
    - you just need the last item to be resolved to check if counter = 0, then you can resolve the entire result

In [ ]:
/**
 * @param {Array} iterable
 * @return {Promise<Array>}
 * 
 * returns a single promise that resolves to an array of results from input
 * 
 * that promise will only resolve when all of the input's promises have resolved
 * or if input iterable has no promiseshttp://localhost:8888/notebooks/OneDrive/Desktop/Great-Front-End/JavaScript-Polyfills/Medium.ipynb#
 * 
 * rejects immediately if any of the inputs reject or throw an error
 * will reject with first rejection message/error
 * 
 * will probably have to use .then() for error handling
 * 
 */
export default function promiseAll(iterable) {
  // how do we know if all of them resolved?
  // will probably need a counter equal to # of iterables
  // if a value resolves right away or is not a promise, we counter--
  // if a promise resolves, we counter-- in .then() block
  // if we have a an error, we reject immediately

  // how do we check if all of them are done?
  // could have a while loop that continually checks if counter === 0
  // and then we resolve the resulting array
  return new Promise((resolve, reject) => {
    let counter = iterable.length;
    const result = new Array(counter);
    if (counter === 0) {
      resolve(result);
      return;
    }

    iterable.forEach(async(element, i) => {
      try {
        const value = await element;
        result[i] = value;
        counter--;
        if (counter === 0) {
          resolve(result);
        }
      } catch(e) {
        reject(e);
      }
    });
  });
}

## Array.prototype.concat()

* pretty straightforward
* just have to remember that anytime you're dealing with prototype method implementations, you want to use `this` to access the object
* `Array.isArray()` is extremely useful when you might have to handle nested arrays
* `spread syntax (...)` is an invaluable tool

In [ ]:
/**
 * @template T
 * @param {...(T | Array<T>)} items
 * @return {Array<T>}
 * 
 * does not mutate arrays, returns NEW array
 * 
 * if passed in as array, must spread them into result
 *  - what about nested arrays?
 *  - yes, must also handle nested arrays but by how much????
 *  - only the top level
 *  - e.g. [1, 2, 3].concat(4, [5, 6]) => [1, 2, 3, 4, 5, 6]
 *  - [1, 2, 3].concat(4, [[5], 6]) => [1, 2, 3, 4, [5], 6]
 * 
 * if passed in as list of arguments, must also push them into new array
 * 
 * must also handle sparse arrays
 *  - probably want to push undefined
 */
Array.prototype.myConcat = function (...items) {
  const result = [...this];
  // items = array of arguments

  // loop through each item
  for (const item of items) {
    // if item is an array, use spread operator to and push into result
    if (Array.isArray(item)) {
      result.push(...item);
    } else {
      result.push(item);
    }
  }

  // else, just push into result

  // return new array
  return result;
};